In [55]:
import numpy as np
import pandas as pd 
import os

In [56]:
import numpy as np 
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Dropout,Flatten,Input,InputLayer,GlobalAveragePooling1D,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

In [57]:
from sklearn.model_selection import train_test_split
from shutil import copyfile

mainDataDir='/kaggle/input/multi-cancer/Multi Cancer/Kidney Cancer'
kidneyNormalDir=os.path.join(mainDataDir,'kidney_normal')
kidneyTumorDir=os.path.join(mainDataDir,'kidney_tumor')

In [58]:
trainDir = '/kaggle/working/training_data'
valDir = '/kaggle/working/validation_data'

In [59]:
os.makedirs(trainDir,exist_ok=True)
os.makedirs(valDir,exist_ok=True)

In [60]:
def split_and_copy(class_dir, train_output_dir, val_output_dir):
    images = [os.path.join(class_dir, img) for img in os.listdir(class_dir) if img.endswith(('jpg', 'jpeg', 'png'))]

    train_images, val_images = train_test_split(images, test_size=0.25, random_state=40)

    os.makedirs(train_output_dir, exist_ok=True)
    os.makedirs(val_output_dir, exist_ok=True)

    for img in train_images:
        copyfile(img, os.path.join(train_output_dir, os.path.basename(img)))

    for img in val_images:
        copyfile(img, os.path.join(val_output_dir, os.path.basename(img)))

split_and_copy(kidneyNormalDir,os.path.join(trainDir,'kidney_normal'),os.path.join(valDir,'kidney_normal'))
split_and_copy(kidneyTumorDir,os.path.join(trainDir,'kidney_tumor'),os.path.join(valDir,'kidney_tumor'))

In [61]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory(
    '/kaggle/working/training_data',
    target_size=(224,224),
    class_mode='binary',
    batch_size=64
)

val_generator=val_datagen.flow_from_directory(
    '/kaggle/working/validation_data',
    target_size=(224,224),
    class_mode='binary',
    batch_size=64
)

Found 7500 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


In [62]:
print(train_generator.class_indices)
print(val_generator.class_indices)

{'kidney_normal': 0, 'kidney_tumor': 1}
{'kidney_normal': 0, 'kidney_tumor': 1}


In [63]:
print(train_generator.classes)  
print(val_generator.classes)  

[0 0 0 ... 1 1 1]
[0 0 0 ... 1 1 1]


In [64]:
train_generator.class_indices

{'kidney_normal': 0, 'kidney_tumor': 1}

## Training the data

In [65]:
noOfClasses=len(train_generator.class_indices)
noOfClasses

2

In [66]:
imageShape=(224,224,3)

In [67]:
from tensorflow.keras.regularizers import l2,l1

model = Sequential([
    Input(shape=imageShape),
    Conv2D(32,(3,3),activation='relu',kernel_regularizer=l2(0.001)),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.2),
    Flatten(),
    Dense(128,activation='relu'),
    Dense(noOfClasses,activation='sigmoid')
])

In [68]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=0.001)  # Lower learning rate
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [69]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 109, 109, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 93312)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │    11,944,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,954,466 (45.60 MB)

 Trainable params: 11,954,466 (45.60 MB)

 Non-trainable params: 0 (0.00 B)

In [70]:
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

In [71]:
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.4,
                              patience=5, min_lr=0.01)

In [72]:
batch_size=16
steps_per_epoch = len(train_generator) // batch_size  
validation_steps = len(val_generator) // batch_size  
model.fit(train_generator, validation_data=val_generator, epochs=100, callbacks=[early_stopping],steps_per_epoch=steps_per_epoch,validation_steps=validation_steps)

Epoch 1/100


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


7/7 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - accuracy: 0.4849 - loss: 2.1777 - val_accuracy: 0.4844 - val_loss: 1.2256
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 17s 3s/step - accuracy: 0.4745 - loss: 0.9842 - val_accuracy: 0.5547 - val_loss: 0.6756
Epoch 3/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.5416 - loss: 0.6912 - val_accuracy: 0.7266 - val_loss: 0.6715
Epoch 4/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 17s 3s/step - accuracy: 0.5205 - loss: 0.6903 - val_accuracy: 0.6875 - val_loss: 0.6496
Epoch 5/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.5974 - loss: 0.6797 - val_accuracy: 0.6250 - val_loss: 0.6385
Epoch 6/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 17s 3s/step - accuracy: 0.5867 - loss: 0.6625 - val_accuracy: 0.7109 - val_loss: 0.5813
Epoch 7/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.7040 - loss: 0.6374 - val_accuracy: 0.6797 - val_loss: 0.6059
Epoch 8/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 17s 3s/step - accuracy: 0.7128 - loss: 0.5814 - val_accuracy: 0.6719 - val_loss: 0.5705
Epoch 9/100


In [73]:
from keras.models import load_model

In [74]:
# Evaluate on the training generator
train_results = model.evaluate(train_generator,batch_size=32)
print("Train loss, Train accuracy:", train_results)

# Evaluate on the validation generator
val_results = model.evaluate(val_generator)
print("Validation loss, Validation accuracy:", val_results)

118/118 ━━━━━━━━━━━━━━━━━━━━ 143s 1s/step - accuracy: 0.7751 - loss: 0.4892
Train loss, Train accuracy: [0.49790510535240173, 0.7725333571434021]
40/40 ━━━━━━━━━━━━━━━━━━━━ 24s 586ms/step - accuracy: 0.7465 - loss: 0.5289
Validation loss, Validation accuracy: [0.5231881141662598, 0.7483999729156494]


In [75]:
def savingModel(model, model_name="model"): 
    model_filename = f"{model_name}.h5"
    model.save(model_filename)
    print(f"Model saved as {model_filename}")

In [86]:
savingModel(model,model_name="kidneyCancer")

Model saved as kidneyCancer.h5


In [87]:
def loadingModel(model_name="model"): 
    try:
        model = load_model(f"{model_name}.h5")
        model.summary() 
        return model 
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

In [88]:
loadedModel=loadingModel("kidneyCancer")

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 109, 109, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 93312)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │    11,944,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,954,468 (45.60 MB)

 Trainable params: 11,954,466 (45.60 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [89]:
def loadImg(imgPath): 
    img = tf.keras.preprocessing.image.load_img(imgPath, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array=img_array/255.0
    return img_array

In [90]:
import tensorflow as tf

In [91]:
def predictImagesInFolder(testFolder, class_name, model_name): 
    savedModel = loadingModel(model_name)
    print(savedModel)

    if savedModel is None:
        print("Model loading failed.")
        return

    for subdir, dirs, files in os.walk(testFolder): 
        if subdir == testFolder: 
            continue
        trueLabel = os.path.basename(subdir)
        for file in files: 
            img_path = os.path.join(subdir, file)
            img = loadImg(img_path)
            
            # Debugging prediction probabilities
            predictions = savedModel.predict(img)
#             print(f"Prediction Probabilities for {file}: {predictions}")

            predicted_class_index = np.argmax(predictions, axis=1)
            predicted_class = class_name[predicted_class_index[0]]
            print(f"Predicted class: {predicted_class} File: {file}")

In [92]:
def testFolder(folder): 
    testFolder=f"{folder}"
    return testFolder

In [93]:
kidneyCancerTestFolder=testFolder("/kaggle/working/validation_data")

In [94]:
kidneyCancerClassname={0:'kidney_normal', 1:'kidney_tumor'}

In [ ]:
predictImagesInFolder(kidneyCancerTestFolder,kidneyCancerClassname,model_name="kidneyCancer")

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 109, 109, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 93312)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │    11,944,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,954,468 (45.60 MB)

 Trainable params: 11,954,466 (45.60 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

<Sequential name=sequential_4, built=True>
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Predicted class: kidney_normal File: kidney_normal_3620.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Predicted class: kidney_normal File: kidney_normal_3432.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Predicted class: kidney_normal File: kidney_normal_0029.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predicted class: kidney_normal File: kidney_normal_4486.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Predicted class: kidney_normal File: kidney_normal_0218.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predicted class: kidney_normal File: kidney_normal_2848.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predicted class: kidney_normal File: kidney_normal_4194.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Predicted class: kidney_normal File: kidney_normal_1491.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predicted class: kidney_normal File: kidney_normal_0433.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Predicted class: kidney_normal File: 